In [1]:
import backtrader as bt
import datetime
import pandas as pd
import schwabdev
import datetime
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

app_key = os.getenv("APP_KEY")
app_secret = os.getenv("APP_SECRET")

client = client = schwabdev.Client(app_key, app_secret)
data = client.price_history(symbol='NVDA', periodType='year', period=20, frequencyType='daily').json()['candles']
df = pd.DataFrame(data)

df['datetime'] = pd.to_datetime(df['datetime'], unit='ms').dt.strftime('%m-%d-%Y')

print(df.info())

<class 'pandas.DataFrame'>
RangeIndex: 5031 entries, 0 to 5030
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   open      5031 non-null   float64
 1   high      5031 non-null   float64
 2   low       5031 non-null   float64
 3   close     5031 non-null   float64
 4   volume    5031 non-null   int64  
 5   datetime  5031 non-null   str    
dtypes: float64(4), int64(1), str(1)
memory usage: 236.0 KB
None


In [3]:
class BH_strat(bt.Strategy):
    params = (
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f"{dt.isoformat()} | {txt}")
    
    def __init__(self):
        self.dataclose = self.data[0].close

        self.order = None
        self.buycomm = None
        self.buyprice = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f"BUY EXECUTED | Price: {order.executed.price:.2f} | Cost: {order.executed.value:.2f} | Comm: {order.executed.comm:.2f}")
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f"SELL EXECUTED | Price: {order.executed.price:.2f} | Cost: {order.executed.value:.2f} | Comm: {order.executed.comm:.2f}")
        elif order.status in [order.Cancelled, order.Margin, order.Rejected]:
            self.log("ORDER CANCELED / MARGIN / REJECTED")
        
        self.order = None

    def next(self):
        self.log(f"Close: {self.dataclose[0]:.2f}")

        if len(self) == 1:
            self.log(f"BUY CREATED | Price: {self.dataclose[0]}")
            self.order = self.buy()
            
    def stop(self):
        self.log(f'Ending Value: {self.broker.getvalue():.2f}', doprint=True)
